In [19]:
# import the package
from Bio import Entrez

# always give the NCBI your email address
Entrez.email = "bleseapringle@gradcenter.cuny.edu"
handle = Entrez.esearch(db = "nuccore", term = ("Mus musculus[Organism]"), retmax = 500)

In [20]:
record = Entrez.read(handle)
handle.close()
# record is a dictionary, we can look at the keys
record.keys()

dict_keys(['Count', 'RetMax', 'RetStart', 'IdList', 'TranslationSet', 'TranslationStack', 'QueryTranslation'])

The Entrez.read parser breaks the retrieved XML data down into individual parts,
and transforms them into Python objects that can be accessed individually. Let’s see how
many sequences are available in the nucleotide database for our search term, and access the
record IDs (note that NCBI returns only 20 IDs by default to keep traffic on its server low;
if you need all IDs, call Entrez.esearch again and set retmax to the maximum number of
IDs, here, 126):

In [21]:
record["Count"]
print(record)
# retrieve list of genebank identifiers
#id_list = record["IdList"]
#print(id_list)

{'Count': '11125642', 'RetMax': '500', 'RetStart': '0', 'IdList': ['2701800492', '2701800491', '2701800490', '2701800486', '2701800484', '2701800482', '2701800481', '2701800479', '2701800478', '2701800475', '2701800474', '2701800473', '2701800472', '2701800469', '2701800466', '2701800463', '2701800462', '2701800461', '2701800459', '2701800458', '2701800457', '2701800455', '2701800454', '2701800453', '2701800452', '2701800450', '2701800448', '2701800447', '2701800446', '2701800444', '2701800443', '2701800442', '2701800440', '2701800438', '2701800437', '2701800434', '2701800431', '2701800428', '2701800427', '2701800426', '2701800423', '2701800421', '2701800417', '2701800416', '2701800415', '2701800414', '2701800413', '2701800411', '2701800410', '2701800406', '2701800405', '2701800402', '2701800401', '2646291840', '2646291827', '2644775015', '2644607909', '2644607876', '2644607826', '2592805038', '2592805037', '2592805024', '2592805018', '2592805005', '2592804989', '2592804984', '22888742

In [8]:
# data using Entrez.fetch. We retrieve the first ten sequences in fasta format 
# and save them to a file:
Entrez.email = "bleseapringle@gradcenter.cuny.edu"
handle = Entrez.efetch(db = "nuccore", rettype = "fasta", retmode = "text", id = id_list[:128])
# set up a handle to an output file
out_handle = open("Uropsilus_seq.fasta", "w")
# write obtained seq data to file
for line in handle:
    out_handle.write(line)
out_handle.close()
handle.close()
        

4.3.2 Input and output of sequence data using SeqIO
Next, we use the module SeqIO to manipulate our sequences and obtain more information
about our U. investigator results:

In [9]:
from Bio import SeqIO
handle = open("Uropsilus_seq.fasta", "r")
# print ID and seq length
for record in SeqIO.parse(handle, "fasta"):
    print(record.description)
    print(len(record))
handle.close()

NC_060485.1 Uropsilus investigator mitochondrion, complete genome
16519
MW682667.1 Uropsilus investigator voucher KIZ:PM180283 cytochrome b (cytb) gene, complete cds; mitochondrial
1140
KC516837.1 Uropsilus investigator isolate A11 apolipoprotein B (ApoB) gene, partial cds
573
KC516819.1 Uropsilus investigator voucher mlxs331 cytochrome c oxidase subunit I (COI) gene, partial cds, alternatively spliced; mitochondrial
912
KC516818.1 Uropsilus investigator voucher mlxs022 cytochrome c oxidase subunit I (COI) gene, partial cds, alternatively spliced; mitochondrial
912
KC516817.1 Uropsilus investigator voucher CY11N009 cytochrome c oxidase subunit I (COI) gene, partial cds, alternatively spliced; mitochondrial
912
KC516797.1 Uropsilus investigator voucher mlxs331 cytochrome b (CYTB) gene, complete cds; mitochondrial
1140
KC516796.1 Uropsilus investigator voucher mlxs022 cytochrome b (CYTB) gene, complete cds; mitochondrial
1140
KC516795.1 Uropsilus investigator voucher CY11N009 cytochrome 

Let’s select only the records of the BMI1 gene and shorten our sequences before writing to a new file (note you’ll need to repeat the preceding steps but with Entrez.esearch
including retmax set to the maximum number of IDs, i.e., handle = Entrez.esearch(db =
"nuccore", term = ("Uropsilus investigator[Organism]"), retmax = 126), and with
Entrez.efetch also reflecting the maximum number of IDs, i.e., handle = Entrez.efetch(db
= "nuccore", rettype = "fasta", retmode = "text", id = id list[:126])):

In [10]:
import re
output_handle = open("Uropsilus_BMI1.fasta", "w")
for record in SeqIO.parse("Uropsilus_seq.fasta", "fasta"):
    # find BMI1 seqs
    if re.search("BMI1", record.description):
        print(record.id)
    # shorten seq by Python slicing
        short_seq = record[:100]
        #SeqIO.write(short_seq, out_handle, "fasta")
#output_handle.close()

KF778086.1
KF778085.1
KF778084.1
KF778083.1
KF778050.1
KF778049.1
KF778048.1


4.3.3 Programmatic BLAST search
The Basic Local Alignment Search Tool (BLAST) finds regions of similarity between biological sequences. Biopython provides a module to conveniently run a BLAST search against
online databases:

In [11]:
# NCBIWWW allows programmatic access of NCBI's BLAST server
from Bio.Blast import NCBIWWW
# retrieve seqs using SeqIO
handle = open("Uropsilus_BMI1.fasta", "r")
# convert SecRecord into a list for easy access
records = list(SeqIO.parse(handle, "fasta"))
# Example of retrieved (fourth) sequence:
# input: print(records[3].id, " ", records[3].seq)
# output: KF778083.1 TATTATGCTGTTTTGTGAACCTGTAG[...]

We are now ready to run our BLAST search against the NCBI nucleotide database
(other options are blastp, blastx, tblastn, or tblastx):

In [14]:
# always tell NCBI who you are
Entrez.email = "bleseapringle@gradcenter.cuny.edu"
#NCBIWWW.qblast requires three arguments:
#program, database, sequence
# note that this can take more than five minutes to run
result_handle = NCBIWWW.qblast("blastn", "nt", records[3].seq)
# set up output file
save_file = open("my_blast.xml", "w")
# write results to output file
save_file.write(result_handle.read())
save_file.close()
result_handle.close()

IndexError: list index out of range

4.3.4 Querying PubMed for scientific literature information
Last but not least, we query NCBI’s scientific literature database, PubMed. We can, for
instance, obtain information on specific journals, search for authors or specific terms in abstracts and titles (see goo.gl/en5Ej6 for a list and description of all search options). Here,
we want to find the latest information on the gene Spaetzle in Drosophila. We first query
the database and then extract sentences that contain the word Spaetzle:

In [16]:
Entrez.email = "bleseapringle@gradcenter.cuny.edu"
handle = Entrez.esearch(db = "pubmed",
term = ("spaetzle[Title/Abstract] AND Drosophila[ALL]"),
usehistory = "y")
# parse results and convert to Python dictionary
record = Entrez.read(handle)
handle.close()
# see how many hits were found - type: record ["Count"]

In [17]:
record ["Count"]

'18'

In our Entrez.esearch call, we turned the use history option on. This option is recommended for complex or large queries. NCBI keeps track of our activities and we can
reference a previous query up to eight hours later by providing \WebEnv" and \QueryKey"
as parameters in our next E-Utilities call

In [18]:
# store WebEnv and QueryKey in variable for later reference
webenv = record["WebEnv"]
query_key = record["QueryKey"]

Now that we know what’s available, we retrieve data from the PubMed database by calling Entrez.efetch and write the results to a file. The query can be shortened by reusing
our WebEnv and WebKey information:

In [19]:
Entrez.email = "bleseapringle@gradcenter.cuny.edu"
handle = Entrez.efetch(db = "pubmed",
                       rettype = "medline",
                       retmode = "text",
                       webenv = webenv,
                       query_key = query_key)
out_handle = open("Spaetzle_abstracts.txt", "w")
data = handle.read()
handle.close()
out_handle.write(data)
out_handle.close()
                       

Using regular expressions, we extract all sentences containing the word Spaetzle:

In [20]:
import re
with open("Spaetzle_abstracts.txt") as datafile:
    pubmed_input = datafile.read()
    # delete newlines followed by 6 spaces
    # to have titles and abstracts on one line
    pubmed_input = re.sub(r"\n\s{6}", " ", pubmed_input)
    for line in pubmed_input.split("\n"):
        if re.match("PMID", line):
            PMID = re.search(r"\d+", line).group()
        if re.match("AB", line):
            spaetzle = re.findall(r"([^.]*?Spaetzle[^.]*n.)", line)
            # don't print if list of matches is empty
            if spaetzle:
                print("PubMedID: ", PMID, " ", spaetzle)

PubMedID:  36675034   [' Spaetzle proteins, activated by immune  signals from upstream components, bind to Toll proteins, thus, activating the  Toll pathway, which in turn, induces AMP gene']
PubMedID:  32591083   [' These  events promote the ventral processing of Spaetzle, a ligand for Toll, which  ultimately sets up the embryonic dorsal-vent']
PubMedID:  32014469   [' In addition, beta-1, 3-glucan (laminarin), the main  component of the cell wall of the pathogenic fungus Metarhizium acridum, was  capable of activating the Toll signaling pathway (Spaetzle and Cactus) when it  was applied on ']
PubMedID:  31719046   ['  The dynamics indicate that a sharp extracellular gradient is formed through  diffusion-based shuttling of the Spaetzle (Spz) morphogen that progresses through  several nuclear divisions']
PubMedID:  27314646   [' While cytokines activating immune responses,  such as Spaetzle or Unpaired-3, have been identified and characterized in  Drosophila, much less is known regardi

You just retrieved all sentences from titles and abstracts in PubMed that contain the
keywords Spaetzle and Drosophila. Although you can achieve the same task relatively quickly
using the graphical web interface, the programmatic approach is a lot more efficient when
you want to learn about another 20 proteins and possibly repeat the search a year later. By
slightly modifying the script (e.g., with a loop that cycles through your keywords of interest),
you can immediately start reading the information instead of spending your time clicking
through web pages and manually pasting results to different files.